<a href="https://colab.research.google.com/github/fabiormazza/IA025_2022S1/blob/main/ex08/fabio_mazza/Aula_8_Exerc%C3%ADcio_Template2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
nome = "Fabio Renato Zocal Mazza"
print(f'Meu nome é {nome}')

Meu nome é Fabio Renato Zocal Mazza


#  Exercício: Modelo de Linguagem com auto-atenção

Este exercício é similar ao da Aula 7, mas iremos agora treinar uma rede neural *com auto-atenção* para prever a próxima palavra de um texto, data as palavras anteriores como entrada. 

Na camada de auto-atenção, não se esqueça de implementar:
- Embeddings de posição
- Projeções lineares (WQ, WK, WV, WO)
- Conexões residuais
- Camada de feed forward (2-layer MLP)



O dataset usado neste exercício (BrWaC) possui um tamanho razoável e você vai precisar rodar seus experimentos com GPU.

Alguns conselhos úteis:
- **ATENÇÃO:** o dataset é bem grande. Não dê comando de imprimí-lo.
- Durante a depuração, faça seu dataset ficar bem pequeno, para que a depuração seja mais rápida e não precise de GPU. Somente ligue a GPU quando o seu laço de treinamento já está funcionando
- Não deixe para fazer esse exercício na véspera. Ele é trabalhoso.

In [2]:
# iremos utilizar a biblioteca dos transformers para ter acesso ao tokenizador do BERT.
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Importação dos pacotes

In [3]:
import collections
import itertools
import functools
import math
import random

import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm_notebook


In [4]:
# Check which GPU we are using
!nvidia-smi

Wed May 25 18:44:50 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
if torch.cuda.is_available(): 
   dev = "cuda:0"
else: 
   dev = "cpu"
device = torch.device(dev)
print('Using {}'.format(device))

Using cuda:0


## Implementação do MyDataset

In [ ]:
#from typing import List


#def tokenize(text: str, tokenizer):
#    return tokenizer(text, return_tensors=None, add_special_tokens=False).input_ids


#class MyDataset():
#    def __init__(self, texts: List[str], tokenizer, context_size: int):
#        self.examples = []
#        for text in tqdm_notebook(texts):
#            token_ids = tokenize(text=text, tokenizer=tokenizer)
#            if len(token_ids) < context_size + 1:
#                continue
#            # Compute n-grams:
#            for i in range(len(token_ids) - context_size):
#                input_ids = token_ids[i:i + context_size]
#                target_id = token_ids[i + context_size]
#                self.examples.append((input_ids, target_id)) 
#
#    def __len__(self):
#        return len(self.examples)
#
#    def __getitem__(self, idx):
#        input_ids, target_id = self.examples[idx]
#        return torch.LongTensor(input_ids), target_id

In [6]:
from typing import List


def tokenize(text: str, tokenizer):
    return tokenizer(text, return_tensors=None, add_special_tokens=False).input_ids


class MyDataset():
    def __init__(self, texts: List[str], tokenizer, context_size: int):
        self.context_size = context_size
        inputs = []
        targets = []

        for text in tqdm_notebook(texts):
          token = tokenize(text, tokenizer)
          for i in range(len(token) - context_size):
            inputs.append(token[i:i+context_size])
            targets.append(token[i+context_size])
        
        self.inputs = torch.LongTensor(inputs)
        self.targets = torch.LongTensor(targets)

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.targets[idx]

## Testando se a implementação do MyDataset está correta

In [7]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

dummy_texts = ['Eu gosto de correr', 'Ela gosta muito de comer pizza']

dummy_dataset = MyDataset(texts=dummy_texts, tokenizer=tokenizer, context_size=3)
dummy_loader = DataLoader(dummy_dataset, batch_size=6, shuffle=False)
assert len(dummy_dataset) == 5
print('passou no assert de tamanho do dataset')

first_batch_input, first_batch_target = next(iter(dummy_loader))

correct_first_batch_input = torch.LongTensor(
    [[ 3396, 10303,   125],
     [ 1660,  5971,   785],
     [ 5971,   785,   125],
     [  785,   125,  1847],
     [  125,  1847, 13779]])

correct_first_batch_target = torch.LongTensor([13239,   125,  1847, 13779, 15616])

assert torch.equal(first_batch_input, correct_first_batch_input)
print('Passou no assert de input')
assert torch.equal(first_batch_target, correct_first_batch_target)
print('Passou no assert de target')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/2 [00:00<?, ?it/s]

passou no assert de tamanho do dataset
Passou no assert de input
Passou no assert de target


# Carregamento do dataset 

Iremos usar uma pequena amostra do dataset [BrWaC](https://www.inf.ufrgs.br/pln/wiki/index.php?title=BrWaC) para treinar e avaliar nosso modelo de linguagem.

In [8]:
!wget -nc https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula7/sample_brwac.txt

File ‘sample_brwac.txt’ already there; not retrieving.



In [9]:
# Load datasets
context_size = 18

valid_examples = 100
test_examples = 100
texts = open('sample_brwac.txt').readlines()

#print('Truncating for debugging purposes.')
#texts = texts[:500]  

training_texts = texts[:-(valid_examples + test_examples)]
valid_texts = texts[-(valid_examples + test_examples):-test_examples]
test_texts = texts[-test_examples:]

training_dataset = MyDataset(texts=training_texts, tokenizer=tokenizer, context_size=context_size)
valid_dataset = MyDataset(texts=valid_texts, tokenizer=tokenizer, context_size=context_size)
test_dataset = MyDataset(texts=test_texts, tokenizer=tokenizer, context_size=context_size)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/24800 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [10]:
print(f'training examples: {len(training_dataset)}')
print(f'valid examples: {len(valid_dataset)}')
print(f'test examples: {len(test_dataset)}')

training examples: 27452745
valid examples: 81170
test examples: 165826


In [11]:
class LanguageModel(torch.nn.Module):

    def __init__(self, vocab_size, context_size, embedding_dim):
        """
        Implements the Self-attention, decoder-only."

        Args:
            vocab_size (int): Size of the input vocabulary.
            context_size (int): Size of the sequence to consider as context for prediction.
            embedding_dim (int): Dimension of the embedding layer for each word in the context.
        """

        super(LanguageModel, self).__init__()

        self.V = vocab_size
        self.L = context_size
        self.D = embedding_dim

        n_linear_1 = 256
        n_linear_2 = 512
        
        self.embedding_matrix = torch.nn.Embedding(vocab_size, embedding_dim, device=device)
        self.embedding_positional = torch.nn.Embedding(context_size, embedding_dim)

        self.wq = nn.Linear(embedding_dim, embedding_dim, bias = False)
        self.wk = nn.Linear(embedding_dim, embedding_dim, bias = False)
        self.wv = nn.Linear(embedding_dim, embedding_dim, bias = False)
        self.wo = nn.Linear(embedding_dim, embedding_dim, bias = False)


        self.linear_1 = torch.nn.Linear(embedding_dim, n_linear_2, device=device)
        self.linear_out = torch.nn.Linear(n_linear_2, vocab_size, device=device, bias = False)
        self.dropout = torch.nn.Dropout(p=0.20)
        self.relu = torch.nn.ReLU()

    def forward(self, inputs):
        """
        Args:
            inputs is a LongTensor of shape (batch_size, context_size)
            
        Returns:
            logits of shape (batch_size, vocab_size)
        """
        batch_size = inputs.shape[0]
        positional = torch.arange(0, self.L).unsqueeze(0).repeat(batch_size, 1).to(inputs.device)
        
        #print(f'V:{self.V}, L:{self.L}, D:{self.D}')

        X = self.embedding_matrix(inputs)
        #print('X shape:', X.shape)
        X = X + self.embedding_positional(positional)
        #print('X shape:', X.shape)
        Q = self.wq(X[:,-1, :].unsqueeze(dim=1))
        #print('Q shape:', Q.shape)
        K = self.wk(X)
        #print('K shape:', K.shape)
        V = self.wv(X)
        #print('V shape:', V.shape)
        scores = torch.matmul(Q, K.permute(0, 2, 1))/math.sqrt(self.D)
        #print('Scores shape:', scores.shape)
        probs = torch.nn.functional.softmax(scores, dim=-1)
        #print('probs shape:', probs.shape)
        E = torch.matmul(probs, V)
        #print('E shape:', E.shape)
        E = self.wo(E)
        #print('E shape:', E.shape)
        out = self.linear_1(E.view(batch_size,-1))
        out = self.dropout(out)
        out = self.relu(out)
        out = self.linear_out(out).squeeze(dim=1)
        #print('out shape:', out.shape)
        T = 3.0 #temperature
        out = out/T

        return out

## Teste o modelo com um exemplo

In [12]:
model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=64,
).to(device)

sample_train, _ = next(iter(DataLoader(training_dataset)))
sample_train_gpu = sample_train.to(device)
model(sample_train_gpu).shape

torch.Size([1, 29794])

In [13]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of model parameters: {num_params}')

Number of model parameters: 17212160


## Assert da Perplexidade


In [14]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)


def perplexity(logits, target):
    """
    Computes the perplexity.

    Args:
        logits: a FloatTensor of shape (batch_size, vocab_size)
        target: a LongTensor of shape (batch_size,)

    Returns:
        A float corresponding to the perplexity
    """
    #loss = nn.functional.cross_entropy(logits, target, reduction='mean')
    criteria = torch.nn.CrossEntropyLoss()
    loss = criteria(logits, target)
    return torch.exp(loss)


n_examples = 1000

sample_train, target_token_ids = next(iter(DataLoader(training_dataset, batch_size=n_examples)))
sample_train_gpu = sample_train.to(device)
target_token_ids = target_token_ids.to(device)
logits = model(sample_train_gpu)

my_perplexity = perplexity(logits=logits, target=target_token_ids)

print(f'my perplexity:              {int(my_perplexity)}')
print(f'correct initial perplexity: {tokenizer.vocab_size}')

assert math.isclose(my_perplexity, tokenizer.vocab_size, abs_tol=7000)
print('Passou o no assert da perplexidade')

my perplexity:              29787
correct initial perplexity: 29794
Passou o no assert da perplexidade


## Laço de Treinamento e Validação

In [57]:
max_examples = 20_000_000
eval_every_steps = 10000
lr = 3e-4


model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=128,
).to(device)

train_loader = DataLoader(training_dataset, batch_size=64, shuffle=True, drop_last=True)
validation_loader = DataLoader(valid_dataset, batch_size=64)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)


def train_step(input, target):
    model.train()
    model.zero_grad()

    logits = model(input.to(device))
    loss = nn.functional.cross_entropy(logits, target.to(device))
    loss.backward()
    optimizer.step()

    return loss.item()


def validation_step(input, target):
    model.eval()
    logits = model(input)
    loss = nn.functional.cross_entropy(logits, target)

    return loss.item()


train_losses = []
n_examples = 0
step = 0
while n_examples < max_examples:
    for input, target in train_loader:
        loss = train_step(input.to(device), target.to(device)) 
        train_losses.append(loss)
        
        if step % eval_every_steps == 0:
            train_ppl = np.exp(np.average(train_losses))

            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(input.to(device), target.to(device))
                    for input, target in validation_loader]))

            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []

        n_examples += len(input)  # Increment of batch size
        step += 1
        if n_examples >= max_examples:
            break

0 steps; 0 examples so far; train ppl: 29658.31, valid ppl: 29701.07
10000 steps; 640000 examples so far; train ppl: 1897.12, valid ppl: 1630.57
20000 steps; 1280000 examples so far; train ppl: 1596.58, valid ppl: 1525.11
30000 steps; 1920000 examples so far; train ppl: 1507.76, valid ppl: 1454.01
40000 steps; 2560000 examples so far; train ppl: 1455.54, valid ppl: 1402.71
50000 steps; 3200000 examples so far; train ppl: 1406.53, valid ppl: 1349.13
60000 steps; 3840000 examples so far; train ppl: 1360.21, valid ppl: 1300.86
70000 steps; 4480000 examples so far; train ppl: 1297.93, valid ppl: 1259.82
80000 steps; 5120000 examples so far; train ppl: 1261.80, valid ppl: 1219.89
90000 steps; 5760000 examples so far; train ppl: 1211.79, valid ppl: 1171.44
100000 steps; 6400000 examples so far; train ppl: 1175.39, valid ppl: 1156.56
110000 steps; 7040000 examples so far; train ppl: 1159.11, valid ppl: 1127.38
120000 steps; 7680000 examples so far; train ppl: 1135.74, valid ppl: 1114.14
13000

## Avaliação final no dataset de teste


Bonus: o modelo com menor perplexidade no dataset de testes ganhará 0.5 ponto na nota final.

In [58]:
test_loader = DataLoader(test_dataset, batch_size=64)

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(input.to(device), target.to(device))
        for input, target in test_loader
    ]))

print(f'test perplexity: {test_ppl}')

test perplexity: 610.8720957225465


## Teste seu modelo com uma sentença

Escolha uma sentença gerada pelo modelo que ache interessante.

In [65]:
prompt = 'Eu gosto de comer aquela pizza porque ela'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-context_size:]  # Usamos apenas os últimos <context_size> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Eu gosto de comer aquela pizza porque ela a
Eu gosto de comer aquela pizza porque ela a sua
Eu gosto de comer aquela pizza porque ela a sua vida
Eu gosto de comer aquela pizza porque ela a sua vida,
Eu gosto de comer aquela pizza porque ela a sua vida, o
Eu gosto de comer aquela pizza porque ela a sua vida, o que
Eu gosto de comer aquela pizza porque ela a sua vida, o que o
Eu gosto de comer aquela pizza porque ela a sua vida, o que o que
Eu gosto de comer aquela pizza porque ela a sua vida, o que o que o
Eu gosto de comer aquela pizza porque ela a sua vida, o que o que o que
Eu gosto de comer aquela pizza porque ela a sua vida, o que o que o que o
Eu gosto de comer aquela pizza porque ela a sua vida, o que o que o que o que
Eu gosto de comer aquela pizza porque ela a sua vida, o que o que o que o que o
Eu gosto de comer aquela pizza porque ela a sua vida, o que o que o que o que o que
Eu gosto de comer aquela pizza porque ela a sua vida, o que o que o que o que o que o
Eu gosto de com